In [1]:
import tensorflow as tf
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from keras.callbacks import TensorBoard

from tensorflow import keras
from keras.models import Sequential
from keras.layers import *
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

import pickle
import time

tf.compat.v1.disable_eager_execution()

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        validation_split=0.3)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Downloads/kagglecatsanddogs_3367a/PetImages",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="training"
)

valid_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Downloads/kagglecatsanddogs_3367a/PetImages",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="validation"
)

test_generator = test_datagen.flow_from_directory(
    directory=r"/home/trojan/Downloads/kagglecatsanddogs_3367a (copy)/test data",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode="binary",
    shuffle=False,
    seed=42
)

NAME = "Cats-vs-dogs-CNN-64*2-{}".format(int(time.time()))


model = Sequential()

model.add(VGG16(include_top=False, weights='imagenet', input_shape = (224,224, 3)))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

model.load_weights("weights-improvement-09-0.99.hdf5")

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'],
              )


STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

#filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
#callbacks_list = checkpoint

'''model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10,
                    callbacks=[callbacks_list, tensorboard])'''
                    


/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

Found 17500 images belonging to 2 classes.
Found 7498 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


'model.fit_generator(generator=train_generator,\n                    steps_per_epoch=STEP_SIZE_TRAIN,\n                    validation_data=valid_generator,\n                    validation_steps=STEP_SIZE_VALID,\n                    epochs=10,\n                    callbacks=[callbacks_list, tensorboard])'

In [2]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 256)         1179904   
_________________________________________________________________
activation_1 (Activation)    (None, 7, 7, 256)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 256)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 256)         590080    
_________________________________________________________________
activation_2 (Activation)    (None, 3, 3, 256)         0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 256)        

In [3]:
from keras.models import load_model 
model.save("first_model.h5")

In [2]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_VALID)

[0.12053237855434418, 0.9839743375778198]

In [3]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

12500/12500 [==============================] - 93s 7ms/step


In [13]:
import numpy as np
import pandas as pd
# Get classes by np.round
cl = np.round(pred)
# Get filenames (set shuffle=false in generator is important)
#filenames=test_generator.filenames

idx_to_cls = {v: k for k, v in train_generator.class_indices.items()}
preds_cls = np.vectorize(idx_to_cls.get)(cl[:,0])
filenames_to_cls = list(zip(test_generator.filenames))




In [14]:
# Data frame
#filenames_to_cls = list(range(1, len(test_generator) + 1))
#filenames_to_cls.sort(key=lambda x: str(x))

results=pd.DataFrame({ "file":filenames_to_cls,"pr":pred[:,0], "class": cl[:,0], "class_name": preds_cls})

results.to_csv("results9.csv",index=False)